In [4]:
from geopy.geocoders import Nominatim
import json
import folium
import requests

## Load Clubs data

In [61]:
with open("../scraper/data/clubs.json", "r") as in_file:
    clubs = json.load(in_file)

In [62]:
len(clubs)

418

In [68]:
clubs[0]["href"]

'/chelsea-fc/kader/verein/631/saison_id/2017'

In [69]:
'''
clubDict = {}
for club in clubs:
    club_id = club["href"].split("/")[4]
    if club_id not in clubDict:
        clubDict[club_id] = club
        
with open("../scraper/data/dictClubs.json", "w") as out:
    json.dump(clubDict, out)
'''

In [73]:
with open("../scraper/data/dictClubs.json", "r") as in_file:
    clubDict = json.load(in_file)

## Retrieve missing coordinates using GoogleMap requests

In [79]:
GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

# For each club, we use its name and the stadium name to make a GoogleMap query and retrieve the stadium's coordinates
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        
        params = {
            'address': club["stadium"] + " " + club["name"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

In [ ]:
# For each club, we use its name and the stadium name to make a GoogleMap query and retrieve the stadium's coordinates
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        
        params = {
            'address': club["stadium"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

### Manual clean up

In [80]:
# Check the clubs for which the coordinates couldn't be retrieved
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        print(str(club_id) + ": " + club["name"])

1245: KAS Eupen
189: Club Atlético Boca Juniors
415: FC Toulouse
7633: UNAM Pumas
5219: AOK Kerkyra
18303: FK Oleksandria
6890: Medipol Basaksehir
2381: Demir Grup Sivasspor
3711: CD Cruz Azul
626: New England Revolution
660: Shakhtar Donetsk
128: AO Xanthi
309: Paraná Clube
19771: Teleset Mobilya Akhisarspor
26459: NK Veres Rivne
16704: FK Krasnodar
7975: FC Vysocina Jihlava
2477: Karpaty Lviv
29: Everton FC
17467: Cortuluá
1804: Deportivo Toluca
2722: SPAL 2013
2241: Kashima Antlers
7097: CA Unión Santa Fé
2125: Esporte Clube Vitória
1095: ES Troyes AC
1465: GD Estoril Praia
1403: Santos Laguna
4064: Figueirense Futebol Clube
2672: APO Levadiakos
197: AC Sparta Praha
2035: Avaí Futebol Clube (SC)
449: Trabzonspor
1108: Deportivo Alavés
14583: CD Atlético Huila
1286: Club Atlético Newell's Old Boys
132: NAC Breda
2288: Swansea City
398: SS Lazio
6992: Chornomorets Odessa
979: Moreirense FC
7452: Omiya Ardija
3729: Arsenal Tula
1326: Chongqing Dangdai Lifan
19789: Evkur Yeni Malatyaspo

In [149]:
# Change manually the three stadium that couldn't be found with OpenStreetMap nor GoogleMaps
# We then rerun the google map coordinate retrieval
clubs[93]["stadium"] = 'S.B.V. Excelsior'
clubs[170]["stadium"] = 'Başakşehir Fatih Terim Stadium'
clubs[208]["stadium"] = 'Lokeren Daknamstadion'
clubs[209]["stadium"] = 'KAS Eupen'
clubs[221]["stadium"] = 'Obolon Arena'
clubs[235]["stadium"] = 'FC Vysočina Jihlava'
clubs[20]["stadium"] = "Juventus Stadium"

In [58]:
for i in range(len(clubs)):
    if 'Club Le' in clubs[i]["name"]:
        print(i)

261
358


In [60]:
clubs[358]

{'href': '/club-leon-fc/kader/verein/4941/saison_id/2017',
 'market_value': '33,95 Mill. €',
 'name': 'Club León FC',
 'players': [{'href': '/william-yarbrough/profil/spieler/215473',
   'id': '215473',
   'name': 'William Yarbrough'},
  {'href': '/carlos-rodriguez/profil/spieler/102699',
   'id': '102699',
   'name': 'Carlos Rodríguez'},
  {'href': '/guillermo-burdisso/profil/spieler/104233',
   'id': '104233',
   'name': 'Guillermo Burdisso'},
  {'href': '/diego-novaretti/profil/spieler/87493',
   'id': '87493',
   'name': 'Diego Novaretti'},
  {'href': '/miguel-herrera/profil/spieler/234231',
   'id': '234231',
   'name': 'Miguel Herrera'},
  {'href': '/andres-mosquera/profil/spieler/74421',
   'id': '74421',
   'name': 'Andrés Mosquera'},
  {'href': '/juan-ignacio-gonzalez/profil/spieler/71161',
   'id': '71161',
   'name': 'Juan Ignacio González'},
  {'href': '/cristian-torres/profil/spieler/333939',
   'id': '333939',
   'name': 'Cristian Torres'},
  {'href': '/osvaldo-rodriguez/

In [57]:
for i in range(len(clubs)):
    if 'Léon' in clubs[i]["name"]:
        print(i)

In [150]:
params = {
    'address': clubs[201]["stadium"] + " " + clubs[201]["name"]
}
# Do the request and get the response data
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()
# Use the first result
if len(res['results']) > 0:
    result = res['results'][0]
    clubs[201]["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

## TODO: Move slighly clubs which plays in the same stadium so we can see them both on the map

## Save/Load Clubs with location data

In [160]:
'''
with open("../scraper/data/clubs_with_location.json", "w") as out:
    json.dump(clubs, out)
'''

'\nwith open("../scraper/data/clubs_with_location.json", "w") as out:\n    json.dump(clubs, out)\n'

In [5]:
with open("../scraper/data/clubs_with_location.json", "r") as in_file:
    clubs = json.load(in_file)

# Display clubs on map

In [46]:
m = folium.Map(location=[52, 15], zoom_start=4)
from folium.plugins import MarkerCluster

In [51]:
marker_cluster = MarkerCluster([]).add_to(m)

In [52]:
for club in clubs:
    folium.Marker(
        location=club["position"],
        popup=club["name"],
        icon=None,
    ).add_to(marker_cluster)

In [53]:
m

In [54]:
m.save('index.html')


In [14]:
html="""
    <h1> This is a big popup</h1><br>
    With a few lines of code...
    <p>
    <code>
        from numpy import *<br>
        exp(-2*pi)
    </code>
    </p>
    """
iframe = folium.element.IFrame(html=html, width=500, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.Marker([30,-100], popup=popup, clustered_marker = True).add_to(m)

In [15]:
m

In [35]:
icon = folium.features.CustomIcon("https://seeklogo.com/images/F/FC_Barcelona-logo-D941E13B46-seeklogo.com.png",
                                          icon_size=(24, 24))
marker = folium.Marker([0, 0], icon=icon, popup="FC Barcelona").add_to(m)

In [36]:
m.add_children(marker)

In [38]:
url = 'http://leafletjs.com/examples/custom-icons/{}'.format
icon_image = url('leaf-red.png')
shadow_image = url('leaf-shadow.png')

icon = folium.features.CustomIcon(
    icon_image,
    icon_size=(38, 95),
    icon_anchor=(22, 94),
    shadow_image=shadow_image,
    shadow_size=(50, 64),
    shadow_anchor=(4, 62),
    popup_anchor=(-3, -76)
)

marker = folium.Marker(
    location=[45.3288, -121.6625],
    icon=icon,
    popup='Mt. Hood Meadows'
)


m.add_child(marker)